# Inference Demo - Driver Drowsiness Detection

This notebook demonstrates how to load a trained model and run inference on sample images.


In [ ]:
import sys
from pathlib import Path
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Add src to path
PROJECT_ROOT = Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT / "src"))

from src.backend.models import load_model, predict_image
from src.config.settings import MODEL_PATH, MODEL_INPUT_SIZE

print(f"Project root: {PROJECT_ROOT}")
print(f"Model path: {MODEL_PATH}")


In [ ]:
# Load model
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

try:
    model = load_model(MODEL_PATH, device=device)
    print("✓ Model loaded successfully")
except FileNotFoundError:
    print("❌ Model not found. Please train the model first using notebook 02_model_training.ipynb")
    raise


In [ ]:
# Preprocess function (same as in backend)
def preprocess_image(image: Image.Image) -> torch.Tensor:
    """Preprocess an image for model inference."""
    image = image.resize(MODEL_INPUT_SIZE, Image.Resampling.BILINEAR)
    if image.mode != "RGB":
        image = image.convert("RGB")
    
    img_array = np.array(image).astype(np.float32) / 255.0
    img_tensor = torch.from_numpy(img_array).permute(2, 0, 1)
    
    mean = torch.tensor([0.485, 0.456, 0.406])
    std = torch.tensor([0.229, 0.224, 0.225])
    img_tensor = (img_tensor - mean.view(3, 1, 1)) / std.view(3, 1, 1)
    
    return img_tensor


In [ ]:
# Test on sample images
DATA_DIR = PROJECT_ROOT / "Data"
drowsy_samples = list((DATA_DIR / "Drowsy").glob("*.png"))[:4]
non_drowsy_samples = list((DATA_DIR / "Non Drowsy").glob("*.png"))[:4]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i, img_path in enumerate(drowsy_samples):
    image = Image.open(img_path).convert("RGB")
    img_tensor = preprocess_image(image)
    
    predicted_class, confidence, probabilities = predict_image(model, img_tensor, device)
    
    axes[0, i].imshow(image)
    axes[0, i].set_title(f"Predicted: {predicted_class}\nConfidence: {confidence:.3f}")
    axes[0, i].axis("off")

for i, img_path in enumerate(non_drowsy_samples):
    image = Image.open(img_path).convert("RGB")
    img_tensor = preprocess_image(image)
    
    predicted_class, confidence, probabilities = predict_image(model, img_tensor, device)
    
    axes[1, i].imshow(image)
    axes[1, i].set_title(f"Predicted: {predicted_class}\nConfidence: {confidence:.3f}")
    axes[1, i].axis("off")

plt.tight_layout()
plt.show()

print("✓ Inference demo complete!")
